# Baseball Data Script

- The purpose of this notebook is to experiment with baseball data scraping scripts. pyBaseball will be used to fetch the data, which will ultimately land in my personal sql server. 
- Every MLB team is needed
- Statcast data along with advanced stats should be scraped
- Tables should be named by stat category and date scraped. 
- The goal is to have time sliced MLB data.
- Frequency of data scraping yet to be determined.

In [2]:
from datetime import date
from datetime import datetime
from configparser import ConfigParser
import pandas as pd
import pybaseball
from pybaseball import statcast
import psycopg2
from sqlalchemy import create_engine
from datetime import date

today = date.today()
st = "statcast_{}".format(today)
st

'statcast_2021-10-26'

In [3]:
# use config parser for security purposes

parser = ConfigParser()
parser.read('nb.ini')
conn_string = parser.get('my_db', 'conn_string')
engine = create_engine(conn_string)

In [3]:
current_year = datetime.now().year
current_year

2021

### Statcast Data
- This data can be overwritten when writing to the sql database. Current table in SQL has 2019, 2020 and up to date 2021 statcast data.

In [4]:
# scrape statcast data

pybaseball.cache.enable()
statcast_2019 = statcast('2019-03-31', '2019-11-30')
statcast_2019['year'] = 2019

This is a large query, it may take a moment to complete


  0%|          | 0/214 [00:00<?, ?it/s]

Skipping offseason dates


100%|██████████| 214/214 [00:24<00:00,  8.81it/s]


In [5]:
statcast_2019.to_csv('F:/datasets/statcast_2019.csv')

In [4]:
statcast_2019 = pd.read_csv('F:/datasets/statcast_2019.csv')

In [5]:
statcast_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732071 entries, 0 to 732070
Data columns (total 94 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Unnamed: 0                       732071 non-null  int64  
 1   pitch_type                       726981 non-null  object 
 2   game_date                        732071 non-null  object 
 3   release_speed                    725452 non-null  float64
 4   release_pos_x                    725431 non-null  float64
 5   release_pos_z                    725431 non-null  float64
 6   player_name                      732071 non-null  object 
 7   batter                           732071 non-null  int64  
 8   pitcher                          732071 non-null  int64  
 9   events                           185858 non-null  object 
 10  description                      732071 non-null  object 
 11  spin_dir                         0 non-null       float64
 12  sp

This data can be cleaned a bit before importing to improve performance. Drop all deprecated features to start.

In [6]:
statcast_2019.to_sql('statcast_2019', engine, if_exists='replace', chunksize= 50, method='multi')

In [4]:
pybaseball.cache.enable()
statcast_2018 = statcast('2018-03-31', '2018-11-30')

This is a large query, it may take a moment to complete


  0%|          | 0/212 [00:00<?, ?it/s]

Skipping offseason dates


100%|██████████| 212/212 [06:06<00:00,  1.73s/it]


In [5]:
statcast_2018.to_sql('statcast_2018', engine, if_exists='replace', chunksize= 50, method='multi')

In [6]:
pybaseball.cache.enable()
statcast_2017 = statcast('2017-03-31', '2017-11-30') 

This is a large query, it may take a moment to complete


  0%|          | 0/214 [00:00<?, ?it/s]

Skipping offseason dates
Skipping offseason dates


100%|██████████| 214/214 [05:46<00:00,  1.62s/it]


In [7]:
statcast_2017.to_sql('statcast_2017', engine, if_exists='replace', chunksize= 100, method='multi')

In [8]:
pybaseball.cache.enable()
statcast_2016 = statcast('2016-03-31', '2016-11-30')

This is a large query, it may take a moment to complete


  0%|          | 0/214 [00:00<?, ?it/s]

Skipping offseason dates
Skipping offseason dates


100%|██████████| 214/214 [05:41<00:00,  1.60s/it]


In [ ]:
statcast_2016.to_sql('statcast_2016', engine, if_exists='replace', chunksize= 100, method='multi')

In [4]:
pybaseball.cache.enable()
statcast_2020 = statcast('2020-03-31', '2020-11-30')
statcast_2020['season'] = 2020

This is a large query, it may take a moment to complete


  0%|          | 0/97 [00:00<?, ?it/s]

Skipping offseason dates
Skipping offseason dates


100%|██████████| 97/97 [02:17<00:00,  1.41s/it]


In [5]:
statcast_2020.to_sql('statcast_2020', engine, if_exists='replace', chunksize= 100, method='multi')

In [ ]:
str(date.today())

'2021-08-09'

In [6]:
opening_day = '{}-03-31'.format(current_year)
opening_day

'2021-03-31'

In [4]:
# this will scrape statcast data dynamically with dates automatically set

today = str(date.today())
opening_day = '{}-03-31'.format(current_year)
opening_day
statcast_2021 = statcast(opening_day, today)
statcast_2021.head()

This is a large query, it may take a moment to complete


100%|██████████| 168/168 [00:31<00:00,  5.37it/s]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
1964,SI,2021-09-13,90.4,-2.44,4.25,"Garza Jr., Ralph",596142,621248,single,hit_into_play,...,5,5,6,6,5,Infield shift,Standard,250,0.301,0.668
2062,CU,2021-09-13,77.2,-2.49,4.15,"Garza Jr., Ralph",596142,621248,None,called_strike,...,5,5,5,5,5,Infield shift,Standard,81,0.0,-0.04
2127,CU,2021-09-13,78.2,-2.31,4.3,"Garza Jr., Ralph",596142,621248,None,ball,...,5,5,5,5,5,Infield shift,Standard,83,0.0,0.028
2162,CU,2021-09-13,77.2,-2.53,4.24,"Garza Jr., Ralph",570482,621248,strikeout,foul_bunt,...,5,5,5,5,5,Standard,Standard,81,-0.108,-0.322
2258,SI,2021-09-13,89.1,-2.2,4.46,"Garza Jr., Ralph",570482,621248,None,called_strike,...,5,5,5,5,5,Strategic,Standard,254,0.0,-0.08


In [5]:
# this works, but very slow, takes about 30 minutes - can probably cut down on the columns
# when finalizing, could parquet files be utilized?
# for advanced hitting stats, append the data (determine frequency, weekly?)
# for statcast, just replace

statcast_2021.to_sql('statcast_2021', engine, if_exists='replace', chunksize=200, method='multi')

In [4]:
from pybaseball import chadwick_register

# get the playerid data
# this should be a one time upload to the database per season(?) - these shouldn't change often
# perhaps a monthly cadence would work well with this table?

data = chadwick_register()

In [5]:
data

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,Aardsma,David,430911,aardd001,aardsda01,1902,2004.0,2015.0
1,Aaron,Hank,110001,aaroh101,aaronha01,1000001,1954.0,1976.0
2,Aaron,Tommie,110002,aarot101,aaronto01,1000002,1962.0,1971.0
3,Aase,Don,110003,aased001,aasedo01,1000003,1977.0,1990.0
4,Abad,Andy,407577,abada001,abadan01,506,2001.0,2006.0
...,...,...,...,...,...,...,...,...
24083,Zupo,Frank,124788,zupof101,zupofr01,1014464,1957.0,1961.0
24084,Zuvella,Paul,124789,zuvep001,zuvelpa01,1014465,1982.0,1991.0
24085,Zuverink,George,124790,zuveg101,zuverge01,1014466,1951.0,1959.0
24086,Zwilling,Dutch,124791,zwild101,zwilldu01,1014467,1910.0,1916.0


In [10]:
data.to_sql('player_lookup', engine, if_exists='replace')

In [42]:
# get advanced hitting data into the database - that one will take some cleaning/filtering
# once tested and validated, create the script

from pybaseball import batting_stats, pitching_stats

# hitting = pybaseball.batting_stats(2018, 2021)
# this code works, for whatever reason pybaseball is mixed up, but
# below code has been confirmed

hitting = batting_stats(current_year, current_year, qual=1)

# note that there is a bug in the pybaseball code that is causing the fangraph pitcher data to be missing
# may have to edit the pybaseball code to fix the issue
# https://github.com/jldbc/pybaseball/issues/221

# pitching = pitching_stats(2021)

In [44]:
# for advanced stats, having a pull date would be useful
# this way we can capture snapshots of performance through time

hitting['date_pull'] = date.today() 

In [45]:
# remove special characters from column names for postgresql requirements

hitting.columns = hitting.columns.str.replace('%', 'pct')
hitting.columns = hitting.columns.str.replace(r'[()]', '', regex=True)

In [46]:
hitting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHitpct,Events,CStrpct,CSWpct,xBA,xSLG,xwOBA,date_pull
30,19611,2021,Vladimir Guerrero Jr.,TOR,22,115,422,494,133,77,...,117.4,187,0.552,339,0.131,0.256,0.305,0.594,0.420,2021-08-15
50,17929,2021,Cedric Mullins II,BAL,26,112,432,483,139,86,...,109.3,141,0.405,348,0.174,0.248,0.285,0.457,0.357,2021-08-15
99,12533,2021,Marcus Semien,TOR,30,116,462,514,126,65,...,109.5,147,0.421,349,0.164,0.256,0.238,0.431,0.323,2021-08-15
33,19755,2021,Shohei Ohtani,LAA,26,115,394,453,106,37,...,119.0,144,0.560,257,0.145,0.298,0.282,0.657,0.428,2021-08-15
60,17350,2021,Rafael Devers,BOS,24,113,422,478,120,59,...,114.4,165,0.516,320,0.099,0.244,0.281,0.562,0.390,2021-08-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,15983,2021,Clint Frazier,NYY,26,66,183,218,34,20,...,111.5,49,0.415,118,0.190,0.294,0.205,0.346,0.311,2021-08-15
637,24655,2021,Owen Miller,CLE,24,31,92,100,13,9,...,107.4,17,0.293,58,0.183,0.303,0.181,0.259,0.227,2021-08-15
619,14818,2021,JaCoby Jones,DET,29,36,100,105,17,13,...,107.3,21,0.362,58,0.172,0.348,0.200,0.349,0.255,2021-08-15
559,17468,2021,Kelvin Gutierrez,- - -,26,46,158,168,33,26,...,111.9,52,0.441,118,0.210,0.334,0.213,0.295,0.250,2021-08-15


In [47]:
# note that % symbols need to be removed from column headers for postgresql import
# add a date column with today's date if hitting stats will be scraped daily/weekly

from sqlalchemy import create_engine

hitting.to_sql('hitting', engine, if_exists='append', chunksize= 100, method='multi')

In [48]:
from pybaseball import pitching_stats

pitching = pitching_stats(2021, qual=1)
pitching

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
89,19361,2021,Corbin Burnes,MIL,26,7,4,5.6,2.23,20,...,7.6,7,0.025,114.2,79,0.284,278,0.181,0.346,1.76
126,10310,2021,Zack Wheeler,PHI,31,10,7,5.5,2.56,24,...,8.4,20,0.049,118.6,124,0.302,411,0.157,0.282,2.72
51,10954,2021,Jacob deGrom,NYM,33,7,2,4.9,1.08,15,...,13.7,11,0.066,112.0,61,0.367,166,0.142,0.358,1.54
335,9132,2021,Nathan Eovaldi,BOS,31,10,7,4.1,3.92,23,...,12.7,23,0.059,115.6,135,0.345,391,0.173,0.296,3.39
106,16137,2021,Carlos Rodon,CHW,28,9,5,4.1,2.38,19,...,17.5,16,0.065,114.6,93,0.380,245,0.149,0.304,2.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,16130,2021,David Hess,MIA,27,2,2,-0.8,8.00,14,...,21.8,8,0.133,116.5,30,0.500,60,0.131,0.217,6.20
724,15068,2021,Yency Almonte,COL,27,1,2,-0.8,9.58,34,...,13.7,8,0.084,110.2,30,0.316,95,0.152,0.257,5.66
544,4696,2021,Nate Jones,- - -,35,0,2,-0.9,5.68,20,...,18.4,7,0.117,112.8,27,0.450,60,0.107,0.239,6.04
542,10811,2021,Mike Foltynewicz,TEX,29,2,11,-1.0,5.66,23,...,16.8,39,0.097,116.5,179,0.444,403,0.169,0.247,5.41


In [51]:
pitching['date_pull'] = date.today() 

In [52]:
# remove special characters from column names for postgresql requirements

pitching.columns = pitching.columns.str.replace('%', 'pct')
pitching.columns = pitching.columns.str.replace(r'[()]', '', regex=True)

In [53]:
pitching.to_sql('pitching', engine, if_exists='append', chunksize= 100, method='multi')

In [58]:
pitching.loc[pitching['Name']=='Craig Kimbrel']

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,Barrels,Barrelpct,maxEV,HardHit,HardHitpct,Events,CStrpct,CSWpct,xERA,date_pull
54,6655,2021,Craig Kimbrel,- - -,33,2,3,2.1,1.25,46,...,5,0.064,114.3,28,0.359,78,0.179,0.364,1.95,2021-08-15
